In [17]:
%matplotlib
#%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re
import pandas as pd
import matplotlib.pyplot as plt
import math

pd.options.mode.use_inf_as_na = True

Using matplotlib backend: Qt5Agg


In [25]:
def phrasesEdit(phraseUser, phraseStim):
    phraseUserEnd = list()
    
    for row_ind in range(0, len(phraseUser)):
        if row_ind!= 0 and phraseUser[row_ind][1] == '':
            phraseUserEnd.append(phraseUser[row_ind-1])
    
    # Attempt to remove all elements that are not end phrases - stimulated or user-generated
    
    for row_ind2 in range(0, len(phraseStim)):
        if phraseStim[row_ind2][1] == 'THE EXPERIMENT IS NOW DONE':
            phraseUserEnd.remove(phraseUserEnd[row_ind2])
            phraseStim.remove(phraseStim[row_ind2])
    
    row_ind3 = len(phraseUserEnd)
    while row_ind3 > -1:
        row_ind3 = row_ind3 - 1
        if phraseUserEnd[row_ind3][1] == '':
            
            if row_ind3 < len(phraseStim):
                phraseStim.remove(phraseStim[row_ind3])
                phraseUserEnd.remove(phraseUserEnd[row_ind3])
    
    # if phrases documented have a comma, the said phrase is split and saved into different columns. They need to be joined
    for row_ind4 in range(0, len(phraseStim)):
        if len(phraseStim[row_ind4])>2:
            phraseStim[row_ind4][1] = ''.join(phraseStim[row_ind4][1:])
        
    for row_ind5 in range(0, len(phraseUserEnd)):
        if len(phraseUserEnd[row_ind5])>2:
            phraseUserEnd[row_ind5][1] = ''.join(phraseUserEnd[row_ind5][1:])
        
#     for i in range(len(phraseUserEnd)):
#         print(phraseUserEnd[i], phraseStim[i])
        
    return phraseUserEnd, phraseStim

In [34]:
def ComputeLIXscore(phraseList):
    
    LIXlist = list()
    
    B = 1
    for phrase in phraseList:
        words = phrase.split()
        A = len(words)
        C = 0
        for word in words:
            if len(word) > 6:
                C = C + 1
        
        LIX = A/B + (C*100)/A
            
        if LIX < 35:
            print('Easy')
        elif LIX > 50:
            print('Difficult')
        else:
            print('Medium')
        LIXlist.append(LIX)

    return LIXlist
        

In [36]:
subjName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'
j = 0

for root, dirs, subfolder in os.walk(subjName):
    LetterLookedAtList = list()
    LetterLookedAt = list()
    
    if not dirs:
        
        if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'tb' in root or 'joha' in root:
            continue
            
        userKeys = None
        scratchPad = None
        gazeLog = None
        stimPhrase = None
        
        for file in subfolder:
            
            if 'js' in root:
                if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                    try:
                        fScratchPad = open(root + '\\' + file)
                        readerScratchPad = csv.reader(fScratchPad)
                        scratchPad = list(readerScratchPad)
                        scratchPad.remove(scratchPad[0])
                        #print(scratchPad)
                    except:
                        if fScratchPad is not None:
                            fScratchPad.close()
                        else:
                            print('error in opening the scratchpad log file')
                            
            elif fnmatch.fnmatch(file, 'user_look*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
                        
            elif fnmatch.fnmatch(file, 'PhraseLog*'):
                try:
                    fStimPhrase = open(root + '\\' + file, encoding='utf-8')
                    readerStimPhrase = csv.reader(fStimPhrase)
                    stimPhrase = list(readerStimPhrase)
                    stimPhrase.remove(stimPhrase[0])
                except:
                    if fStimPhrase is not None:
                        fStimPhrase.close()
                    else:
                        print('error in opening the phrase log file')
            elif fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)
                    scratchPad.remove(scratchPad[0])
                    #print(scratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
                            
            elif fnmatch.fnmatch(file, 'GazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8')
                    readerGazeLog = csv.reader(fGazeLog)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0]) # would not matter much even if the first row was not labels
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the gaze log file')
            else:
                continue
            
                # if all these lists exist
            if userKeys is None or stimPhrase is None or scratchPad is None or gazeLog is None:
                continue
            else:
                a = re.compile('(?<=TypingData\\\May[0-9]{2}\\\)(.*)(?=\\\OptiKey)')
                subjName = a.findall(root)[0]
                print(subjName)
                
                stimPhraseList = [phraseKey[1] for phraseKey in stimPhrase]
            
                # Customization of phrases 
                if 'jg' in root:
                    scratchPad.remove(scratchPad[4])
                if 'snk' in root:
                    stimPhrase.remove(stimPhrase[0])
                # for KEA, 10 was asked not to type
                if 'butikken lukker kl. 10' in stimPhraseList:
                    index = stimPhraseList.index('butikken lukker kl. 10')
                    stimPhrase[index][1] = 'butikken lukker kl.'
            
                #print(phraseLog)
                phraseUserEnd, phraseList = phrasesEdit(scratchPad, stimPhrase)
                
                stimPhraseList = [phraseKey[1] for phraseKey in stimPhrase]
                
                # compute LIX score for every trial of this user, and print
                ComputeLIXscore(stimPhraseList)
                print(' ')


akt_MS
Easy
Difficult
Easy
Medium
 
jl_DT
Easy
Difficult
 
KEA_MS
Easy
Easy
Easy
Medium
Medium
 
lone_DT
Easy
Easy
 
mcc_MS
Difficult
Easy
Easy
Easy
Easy
 
MK_DT
Difficult
Difficult
Easy
Easy
 
ok_MS
Medium
Easy
Easy
Easy
Easy
 
pt_DT
Easy
Difficult
Difficult
Difficult
Easy
 
sc_MS
Easy
Easy
 
sh_MS
Easy
Medium
Medium
Easy
Easy
 
ae_DT
Medium
 
hc_MS
Easy
Difficult
Easy
Difficult
Easy
Easy
 
ib_MS
Difficult
Medium
Medium
Easy
Easy
 
km_DT
Easy
Medium
Difficult
Easy
Easy
Easy
 
ma_DT
Easy
Easy
Easy
Medium
Medium
Easy
 
pgba_DT
Easy
Easy
Easy
Medium
Easy
Easy
 
smn_DT
Easy
Medium
Easy
Easy
Easy
Easy
 
eo_DT
Easy
Difficult
 
jek_MS
Easy
Medium
Difficult
Medium
Easy
Easy
 
jg_DT
Easy
Difficult
 
lg_MS
Easy
Medium
Easy
 
lr_MS
Easy
Medium
Easy
Difficult
Easy
Easy
 
mm_MS
Easy
Easy
Easy
Easy
Medium
Difficult
 
snk_DT
Easy
Difficult
Easy
Easy
Easy
 
